In [16]:
pip install opencv-python numpy tensorflow scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [17]:
import cv2
import numpy as np
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split


In [18]:
# Define categories
categories = ["MildDemented", "ModerateDemented", "NonDemented", "VeryMildDemented"]

# Parameters
img_size = 128  # Resize images to 128x128
data_dir = "/kaggle/input/dataset-alzheimer/Alzheimer_s Dataset/train"  # Directory containing folders for each category


In [19]:
# Preprocess images
def load_images(data_dir, categories, img_size):
    data = []
    labels = []

    for i, category in enumerate(categories):
        folder_path = os.path.join(data_dir, category)
        for filename in os.listdir(folder_path):
            img_path = os.path.join(folder_path, filename)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Load image as grayscale
            if img is not None:
                img = cv2.resize(img, (img_size, img_size))  # Resize to consistent size
                data.append(img)
                labels.append(i)  # Use the index of the category as the label

    data = np.array(data).reshape(-1, img_size, img_size, 1) / 255.0  # Normalize and reshape
    labels = to_categorical(labels, num_classes=len(categories))  # One-hot encode labels
    return data, labels


In [20]:
# Load data
data, labels = load_images(data_dir, categories, img_size)

# Split into training and testing
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)


In [21]:
# Define CNN model
def create_model(input_shape):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(len(categories), activation='softmax')  # Output layer with softmax
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model


In [22]:
# Create and train the model
input_shape = (img_size, img_size, 1)  # Grayscale images have 1 channel
model = create_model(input_shape)

# Augment and fit data
datagen = ImageDataGenerator(rotation_range=10, zoom_range=0.1, width_shift_range=0.1, height_shift_range=0.1)
history = model.fit(datagen.flow(X_train, y_train, batch_size=32), epochs=25, validation_data=(X_test, y_test))


Epoch 1/25
128/128 ━━━━━━━━━━━━━━━━━━━━ 10s 49ms/step - accuracy: 0.4458 - loss: 1.1109 - val_accuracy: 0.5063 - val_loss: 1.0411
Epoch 2/25
128/128 ━━━━━━━━━━━━━━━━━━━━ 6s 44ms/step - accuracy: 0.5071 - loss: 1.0273 - val_accuracy: 0.5688 - val_loss: 0.9465
Epoch 3/25
128/128 ━━━━━━━━━━━━━━━━━━━━ 6s 42ms/step - accuracy: 0.5411 - loss: 0.9381 - val_accuracy: 0.5629 - val_loss: 0.9338
Epoch 4/25
128/128 ━━━━━━━━━━━━━━━━━━━━ 6s 43ms/step - accuracy: 0.5568 - loss: 0.9323 - val_accuracy: 0.5727 - val_loss: 0.9236
Epoch 5/25
128/128 ━━━━━━━━━━━━━━━━━━━━ 6s 44ms/step - accuracy: 0.5627 - loss: 0.9079 - val_accuracy: 0.5883 - val_loss: 0.8996
Epoch 6/25
128/128 ━━━━━━━━━━━━━━━━━━━━ 6s 44ms/step - accuracy: 0.5660 - loss: 0.8915 - val_accuracy: 0.5580 - val_loss: 0.9106
Epoch 7/25
128/128 ━━━━━━━━━━━━━━━━━━━━ 6s 43ms/step - accuracy: 0.5610 - loss: 0.8844 - val_accuracy: 0.6078 - val_loss: 0.8654
Epoch 8/25
128/128 ━━━━━━━━━━━━━━━━━━━━ 6s 43ms/step - accuracy: 0.5958 - loss: 0.8590 - val_acc

In [23]:
# Evaluate model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7219 - loss: 0.6204
Test Accuracy: 71.61%


In [24]:
# Predict on a single image
def classify_image(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (img_size, img_size))
    img = img.reshape(1, img_size, img_size, 1) / 255.0
    prediction = model.predict(img)
    category_index = np.argmax(prediction)
    return categories[category_index]


In [25]:
# Example usage
sample_image = "/kaggle/input/dataset-alzheimer/Alzheimer_s Dataset/test/NonDemented/26 (100).jpg"
classification = classify_image(sample_image)
print(f"Predicted category: {classification}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 197ms/step
Predicted category: NonDemented


In [26]:
model.save("alzheimer_model.h5")